In [1]:
!git clone https://github.com/quangster/machine-translation
%cd machine-translation

Cloning into 'machine-translation'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 49 (delta 8), reused 44 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (49/49), 6.37 MiB | 5.94 MiB/s, done.
Resolving deltas: 100% (8/8), done.
/content/machine-translation


In [2]:
!pip install -q gdown
!gdown --folder 1cPdLNnTlsj3N1FE9x6_K608bCAaYaVGM -O data

Retrieving folder contents
Retrieving folder 1OEYSMb7DHvhpHDkyErFfepIjeyDOiDR4 dev
Processing file 1Koyp92dplbh_S_9UW8wvskbzHW6Gb5Zw dev.en
Processing file 1KVzIWM8IUIS_NdWpctOd_l3FIm901e6L dev.vi
Retrieving folder 1FkG-m-LSXaXCrau3yD8s9_f8Llda3KoF test
Processing file 18XurJYc9T8i4JKzGRknNIMzEBD5bLDex test.en
Processing file 1atCidgee403dxm8mAWIXq9mlfdcYSXc_ test.vi
Retrieving folder 1jrfK8TmZghXISDq7JI-LTZyItZRgnEn2 train
Processing file 1jR128Bdo7vyQc1OPBCE6zEz0gXF6eUDY train.en
Processing file 1hKt2ww1-zZHzXRPl_0ijxUxdWK57XKp1 train.vi
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1Koyp92dplbh_S_9UW8wvskbzHW6Gb5Zw
To: /content/machine-translation/data/dev/dev.en
100% 1.42M/1.42M [00:00<00:00, 13.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1KVzIWM8IUIS_NdWpctOd_l3FIm901e6L
To: /content/machine-translation/data/dev/dev.vi
100% 1.89M/1.89M [00:00<00:00, 16.0MB/

In [3]:
!pip install -q underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.2 MB/s eta 0:00:00


In [4]:
from src.data import Vocabulary, EnTokenizer, ViTokenizer, MTDataset
from src.utils.data import read_corpus

from tqdm import tqdm
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
en_vocab = Vocabulary.load('./ckpts/en_vocab.json')
vi_vocab = Vocabulary.load('./ckpts/vi_vocab.json')
print(en_vocab)
print(vi_vocab)

Vocabulary[language=english, size=34687]
Vocabulary[language=vietnamese, size=21681]


In [6]:
def get_dataset():
    vi_tokenizer = ViTokenizer()
    en_tokenizer = EnTokenizer()

    global en_vocab, vi_vocab

    train_en_sents, train_vi_sents = read_corpus("./data", "train")
    train_en_sents = train_en_sents[:100000]
    train_vi_sents = train_vi_sents[:100000]

    train_en_sents = [en_tokenizer.tokenize(sent.lower()) for sent in tqdm(train_en_sents)]
    train_vi_sents = [vi_tokenizer.tokenize(sent.lower()) for sent in tqdm(train_vi_sents)]

    train_dataset = MTDataset(
        inputs=[en_vocab.words2indexes(sent, add_sos_eos=True) for sent in train_en_sents],
        outputs=[vi_vocab.words2indexes(sent, add_sos_eos=True) for sent in train_vi_sents],
        max_length=20,
        padding_idx=en_vocab['<pad>'],
    )

    val_en_sents, val_vi_sents = read_corpus("./data", "dev")
    val_en_sents = [en_tokenizer.tokenize(sent.lower()) for sent in tqdm(val_en_sents)]
    val_vi_sents = [vi_tokenizer.tokenize(sent.lower()) for sent in tqdm(val_vi_sents)]

    val_dataset = MTDataset(
        inputs=[en_vocab.words2indexes(sent, add_sos_eos=True) for sent in val_en_sents],
        outputs=[vi_vocab.words2indexes(sent, add_sos_eos=True) for sent in val_vi_sents],
        max_length=20,
        padding_idx=en_vocab['<pad>'],
    )

    return train_dataset, val_dataset

In [7]:
train_dataset, val_dataset = get_dataset()
len(train_dataset), len(val_dataset)

100%|██████████| 18719/18719 [00:07<00:00, 2635.52it/s]


(46897, 11668)

In [8]:
# double check
X, y = train_dataset[0]
print(en_vocab.indexes2words(X.numpy()))
print(vi_vocab.indexes2words(y.numpy()))

['<sos>', 'it', 'begins', 'with', 'a', 'countdown', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['<sos>', 'câu', 'chuyện', 'bắt', 'đầu', 'với', 'buổi', 'lễ', 'đếm', 'ngược', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


# Your code goes here

In [9]:
import torch
import torch.nn as nn


class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()

        assert hid_dim % n_heads == 0

        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads

        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)

        self.fc_o = nn.Linear(hid_dim, hid_dim)

        self.dropout = nn.Dropout(dropout)

        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

    def forward(self, query, key, value, mask = None):

        batch_size = query.shape[0]

        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]

        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)

        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]

        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)

        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]

        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale

        #energy = [batch size, n heads, query len, key len]

        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)

        attention = torch.softmax(energy, dim = -1)

        #attention = [batch size, n heads, query len, key len]

        x = torch.matmul(self.dropout(attention), V)

        #x = [batch size, n heads, query len, head dim]

        x = x.permute(0, 2, 1, 3).contiguous()

        #x = [batch size, query len, n heads, head dim]

        x = x.view(batch_size, -1, self.hid_dim)

        #x = [batch size, query len, hid dim]

        x = self.fc_o(x)

        #x = [batch size, query len, hid dim]

        return x, attention

In [10]:
import torch
import torch.nn as nn


class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()

        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):

        #x = [batch size, seq len, hid dim]

        x = self.dropout(torch.relu(self.fc_1(x)))

        #x = [batch size, seq len, pf dim]

        x = self.fc_2(x)

        #x = [batch size, seq len, hid dim]

        return x

In [11]:
import torch
import torch.nn as nn


class EncoderLayer(nn.Module):
    def __init__(self,
                 hid_dim,
                 n_heads,
                 pf_dim,
                 dropout,
                 device):
        super().__init__()

        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim,
                                                                     pf_dim,
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_mask):

        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, 1, 1, src len]

        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)

        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))

        #src = [batch size, src len, hid dim]

        #positionwise feedforward
        _src = self.positionwise_feedforward(src)

        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))

        #src = [batch size, src len, hid dim]

        return src


class Encoder(nn.Module):
    def __init__(self,
                 input_dim,
                 hid_dim,
                 n_layers,
                 n_heads,
                 pf_dim,
                 dropout,
                 device,
                 max_length = 100):
        super().__init__()

        self.device = device

        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)

        self.layers = nn.ModuleList([EncoderLayer(hid_dim,
                                                  n_heads,
                                                  pf_dim,
                                                  dropout,
                                                  device)
                                     for _ in range(n_layers)])

        self.dropout = nn.Dropout(dropout)

        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)

    def forward(self, src, src_mask):

        #src = [batch size, src len]
        #src_mask = [batch size, 1, 1, src len]

        batch_size = src.shape[0]
        src_len = src.shape[1]

        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

        #pos = [batch size, src len]

        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))

        #src = [batch size, src len, hid dim]

        for layer in self.layers:
            src = layer(src, src_mask)

        #src = [batch size, src len, hid dim]

        return src

In [12]:
import torch
import torch.nn as nn


class DecoderLayer(nn.Module):
    def __init__(self,
                 hid_dim,
                 n_heads,
                 pf_dim,
                 dropout,
                 device):
        super().__init__()

        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim,
                                                                     pf_dim,
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, tgt, enc_src, tgt_mask, src_mask):

        #tgt = [batch size, tgt len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #tgt_mask = [batch size, 1, tgt len, tgt len]
        #src_mask = [batch size, 1, 1, src len]

        #self attention
        _tgt, _ = self.self_attention(tgt, tgt, tgt, tgt_mask)

        #dropout, residual connection and layer norm
        tgt = self.self_attn_layer_norm(tgt + self.dropout(_tgt))

        #tgt = [batch size, tgt len, hid dim]

        #encoder attention
        _tgt, attention = self.encoder_attention(tgt, enc_src, enc_src, src_mask)

        #dropout, residual connection and layer norm
        tgt = self.enc_attn_layer_norm(tgt + self.dropout(_tgt))

        #tgt = [batch size, tgt len, hid dim]

        #positionwise feedforward
        _tgt = self.positionwise_feedforward(tgt)

        #dropout, residual and layer norm
        tgt = self.ff_layer_norm(tgt + self.dropout(_tgt))

        #tgt = [batch size, tgt len, hid dim]
        #attention = [batch size, n heads, tgt len, src len]

        return tgt, attention


class Decoder(nn.Module):
    def __init__(self,
                 output_dim,
                 hid_dim,
                 n_layers,
                 n_heads,
                 pf_dim,
                 dropout,
                 device,
                 max_length = 100):
        super().__init__()

        self.device = device

        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)

        self.layers = nn.ModuleList([DecoderLayer(hid_dim,
                                                  n_heads,
                                                  pf_dim,
                                                  dropout,
                                                  device)
                                     for _ in range(n_layers)])

        self.fc_out = nn.Linear(hid_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)

    def forward(self, tgt, enc_src, tgt_mask, src_mask):

        #tgt = [batch size, tgt len]
        #enc_src = [batch size, src len, hid dim]
        #tgt_mask = [batch size, 1, tgt len, tgt len]
        #src_mask = [batch size, 1, 1, src len]

        batch_size = tgt.shape[0]
        tgt_len = tgt.shape[1]

        pos = torch.arange(0, tgt_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

        #pos = [batch size, tgt len]

        tgt = self.dropout((self.tok_embedding(tgt) * self.scale) + self.pos_embedding(pos))

        #tgt = [batch size, tgt len, hid dim]

        for layer in self.layers:
            tgt, attention = layer(tgt, enc_src, tgt_mask, src_mask)

        #tgt = [batch size, tgt len, hid dim]
        #attention = [batch size, n heads, tgt len, src len]

        output = self.fc_out(tgt)

        #output = [batch size, tgt len, output dim]

        return output, attention

In [13]:
import torch
import torch.nn as nn


class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder,
                 decoder,
                 src_pad_idx,
                 tgt_pad_idx,
                 device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.tgt_pad_idx = tgt_pad_idx
        self.device = device

    def make_src_mask(self, src):

        #src = [batch size, src len]

        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        #src_mask = [batch size, 1, 1, src len]

        return src_mask

    def make_tgt_mask(self, tgt):

        #tgt = [batch size, tgt len]

        tgt_pad_mask = (tgt != self.tgt_pad_idx).unsqueeze(1).unsqueeze(2)

        #tgt_pad_mask = [batch size, 1, 1, tgt len]

        tgt_len = tgt.shape[1]

        tgt_sub_mask = torch.tril(torch.ones((tgt_len, tgt_len), device = self.device)).bool()

        #tgt_sub_mask = [tgt len, tgt len]

        tgt_mask = tgt_pad_mask & tgt_sub_mask

        #tgt_mask = [batch size, 1, tgt len, tgt len]

        return tgt_mask

    def forward(self, src, tgt):

        #src = [batch size, src len]
        #tgt = [batch size, tgt len]

        src_mask = self.make_src_mask(src)
        tgt_mask = self.make_tgt_mask(tgt)

        #src_mask = [batch size, 1, 1, src len]
        #tgt_mask = [batch size, 1, tgt len, tgt len]

        enc_src = self.encoder(src, src_mask)

        #enc_src = [batch size, src len, hid dim]

        output, attention = self.decoder(tgt, enc_src, tgt_mask, src_mask)

        #output = [batch size, tgt len, output dim]
        #attention = [batch size, n heads, tgt len, src len]

        return output, attention

In [14]:
def initialize_weights(m):
    if hasattr(m, "weight") and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [15]:
def train(epoch, model, loader, optimizer, criterion, clip):
    '''
        Train step
        Parameters:
            epoch (int): the index of current epoch
            model (Seq2Seq): trained NMT model
            loader (DataLoader): train DataLoader
            optimizer (torch.optim)
            criterion (torch.nn)
        Returns:
            epoch_loss / len(loader) (float): average loss of this epoch
    '''
    model.train()
    device = model.device
    epoch_loss = 0

    with tqdm(enumerate(loader), total=len(loader)) as pbar:
        pbar.set_description(f"Epoch {epoch}")
        for i, (src, tgt) in pbar:
            src = src.to(device)
            tgt = tgt.to(device)

            optimizer.zero_grad()
            output, _ = model(src, tgt[:,:-1])

            #output = [batch size, tgt len - 1, output dim]
            #tgt = [batch size, tgt len]

            output_dim = output.shape[-1]

            output = output.contiguous().view(-1, output_dim)
            tgt = tgt[:,1:].contiguous().view(-1)

            #output = [batch size * tgt len - 1, output dim]
            #tgt = [batch size * tgt len - 1]

            loss = criterion(output, tgt)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()

            epoch_loss += loss.item()
            pbar.set_postfix({"loss": epoch_loss/(i+1)})

    return epoch_loss / len(loader)

In [16]:
def evaluate(model, loader, criterion):
    '''
        Evaluate the model
        Parameters:
            model (Seq2Seq): trained NMT model
            loader (DataLoader): test/valid DataLoader
            criterion (torch.nn)
        Returns:
            epoch_loss / len(loader) (float): average loss
    '''
    model.eval()
    device = model.device
    epoch_loss = 0

    with torch.no_grad():

        for i, (src, tgt) in enumerate(loader):

            src = src.to(device)
            tgt = tgt.to(device)

            output, _ = model(src, tgt[:,:-1])

            #output = [batch size, tgt len - 1, output dim]
            #tgt = [batch size, tgt len]

            output_dim = output.shape[-1]

            output = output.contiguous().view(-1, output_dim)
            tgt = tgt[:,1:].contiguous().view(-1)

            #output = [batch size * tgt len - 1, output dim]
            #tgt = [batch size * tgt len - 1]

            loss = criterion(output, tgt)

            epoch_loss += loss.item()

    return epoch_loss / len(loader)

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

enc_voc_size = len(en_vocab)
dec_voc_size = len(vi_vocab)

src_pad_id = en_vocab["<pad>"]
tgt_pad_id = vi_vocab["<pad>"]

d_model = 256
max_len = 20
ffn_hidden = 512
n_heads = 8
n_layers = 3
drop_prob = 0.1

clip = 1.0

In [18]:
enc = Encoder(input_dim=enc_voc_size,
                    hid_dim=d_model,
                    n_layers=n_layers,
                    n_heads=n_heads,
                    pf_dim=ffn_hidden,
                    dropout=drop_prob,
                    device=device,
                    max_length=max_len)

dec = Decoder(output_dim=dec_voc_size,
                hid_dim=d_model,
                n_layers=n_layers,
                n_heads=n_heads,
                pf_dim=ffn_hidden,
                dropout=drop_prob,
                device=device,
                max_length=max_len)

model = Seq2Seq(enc, dec, src_pad_id, tgt_pad_id, device).to(device)

In [19]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)

print(len(train_loader), len(val_loader))

367 92


In [20]:
criterion = nn.CrossEntropyLoss(ignore_index=tgt_pad_id)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

for epoch in range(20):
    train_loss = train(epoch, model, train_loader, optimizer, criterion, clip)
    val_loss = evaluate(model, val_loader, criterion)
    print(f"Epoch: {epoch+1:02} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

Epoch 0: 100%|██████████| 367/367 [00:47<00:00,  7.71it/s, loss=4.49]


Epoch: 01 | Train Loss: 4.4871 | Val Loss: 4.3983


Epoch 1: 100%|██████████| 367/367 [00:49<00:00,  7.47it/s, loss=3.49]


Epoch: 02 | Train Loss: 3.4855 | Val Loss: 4.0929


Epoch 2: 100%|██████████| 367/367 [00:52<00:00,  7.01it/s, loss=3.11]


...

Epoch: 18 | Train Loss: 1.6526 | Val Loss: 4.1116


Epoch 18: 100%|██████████| 367/367 [00:49<00:00,  7.41it/s, loss=1.62]


Epoch: 19 | Train Loss: 1.6168 | Val Loss: 4.1063


Epoch 19: 100%|██████████| 367/367 [00:54<00:00,  6.78it/s, loss=1.59]


Epoch: 20 | Train Loss: 1.5923 | Val Loss: 4.1184


In [21]:
def translate_sentence(sent, model, src_tok, tgt_tok, device, max_len=256):
    '''
        Translate sentence used trained model with autoregression decoding strategy
        Parameters:
            sent (str or list(str)): sentence string or list of tokens of
                                     source language
            src_tok (BaseTokenizer): tokenizer for source language
            tgt_tok (BaseTokenizer): tokenizer for target language
            model (Seq2Seq): trained NMT model
            device (torch.device): cpu or cuda
            max_len (int): maximum number of tokens for each sentence
        Returns:
            pred_tokens (list(str)): list of translated tokens
            attention (Tensor): attention score from the last layer, can be used
                                to create attention alignment matrix
    '''
    global en_vocab, vi_vocab

    model.eval()
    tokens = [token for token in src_tok.tokenize(sent.lower())]

    tokens = en_vocab.words2indexes(tokens, add_sos_eos=True)
    src_tensor = torch.tensor(tokens, dtype=torch.long).unsqueeze(0).to(device)

    src_mask = model.make_src_mask(src_tensor)
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    pred_indexes = [vi_vocab["<sos>"]]

    # Loop to predict next index from list of previous indexes until reach the
    # end-of-string (eos) token or reach the maximum length
    for i in range(max_len):
        pred_tensor = torch.LongTensor(pred_indexes).unsqueeze(0).to(device)
        tgt_mask = model.make_tgt_mask(pred_tensor)
        with torch.no_grad():
            output, attention = model.decoder(pred_tensor, enc_src, tgt_mask, src_mask)
        pred_token = output.argmax(2)[:,-1].item()
        pred_indexes.append(pred_token)
        if pred_token == vi_vocab["<eos>"]:
            break

    pred_tokens = vi_vocab.indexes2words(pred_indexes)

    return pred_tokens, attention

In [22]:
train_dataset[0]

(tensor([    1,    14,  2278,    23,    11, 14547,     4,     2,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 tensor([   1,  248,  118,  155,   62,   21,  653,  934, 1742,  907,    4,    2,
            0,    0,    0,    0,    0,    0,    0,    0]))

In [23]:
vi_tok = ViTokenizer()
en_tok = EnTokenizer()
translate_sentence("Hello world", model, vi_tok, en_tok, device, max_len=30)

(['<sos>', 'xin', 'chào', 'mừng', '!', '<eos>'],
 tensor([[[[0.0097, 0.8633, 0.1002, 0.0268],
           [0.0554, 0.8052, 0.0872, 0.0523],
           [0.0692, 0.4445, 0.3936, 0.0927],
           [0.1086, 0.7410, 0.0925, 0.0578],
           [0.2742, 0.0536, 0.5686, 0.1037]],
 
          [[0.0137, 0.4405, 0.5328, 0.0130],
           [0.0868, 0.6142, 0.1986, 0.1004],
           [0.3024, 0.1860, 0.2952, 0.2163],
           [0.2274, 0.3339, 0.3806, 0.0581],
           [0.3492, 0.2872, 0.2873, 0.0763]],
 
          [[0.0134, 0.7505, 0.1693, 0.0668],
           [0.0277, 0.2613, 0.5625, 0.1485],
           [0.1158, 0.1336, 0.3091, 0.4414],
           [0.2343, 0.3869, 0.1282, 0.2506],
           [0.4945, 0.0257, 0.3231, 0.1566]],
 
          [[0.0073, 0.9510, 0.0344, 0.0073],
           [0.0789, 0.8421, 0.0525, 0.0265],
           [0.5312, 0.1537, 0.2035, 0.1117],
           [0.7839, 0.0709, 0.0062, 0.1390],
           [0.1664, 0.0677, 0.0318, 0.7342]],
 
          [[0.0016, 0.9933, 0.0010, 0.0

In [24]:
torch.save(model.state_dict(), 'transformer.pth')

In [25]:
translate_sentence("are you ", model, vi_tok, en_tok, device, max_len=30)

(['<sos>', 'bạn', 'là', 'người', 'bạn', '.', '<eos>'],
 tensor([[[[2.4702e-02, 6.8987e-01, 2.1174e-01, 7.3688e-02],
           [3.3886e-01, 1.6683e-01, 3.3255e-01, 1.6175e-01],
           [7.1458e-01, 3.6809e-02, 2.2700e-01, 2.1616e-02],
           [8.5895e-01, 8.9712e-03, 8.4375e-02, 4.7701e-02],
           [9.2994e-01, 1.9175e-02, 1.8088e-02, 3.2792e-02],
           [7.0587e-01, 1.6237e-01, 7.6674e-02, 5.5085e-02]],
 
          [[9.4025e-02, 5.7760e-01, 2.4064e-01, 8.7742e-02],
           [6.1075e-01, 1.7830e-01, 3.6227e-02, 1.7472e-01],
           [7.6177e-01, 9.8738e-02, 6.2399e-02, 7.7092e-02],
           [7.8850e-01, 1.3857e-02, 1.3848e-01, 5.9169e-02],
           [9.6511e-01, 1.2167e-02, 7.9292e-03, 1.4798e-02],
           [8.1791e-01, 6.9930e-02, 3.8902e-02, 7.3256e-02]],
 
          [[2.6824e-02, 4.1240e-01, 4.4029e-01, 1.2049e-01],
           [6.5002e-02, 8.3126e-01, 1.5240e-02, 8.8494e-02],
           [3.8886e-01, 6.2735e-02, 4.0598e-01, 1.4243e-01],
           [6.4960e-01, 